# Analise de Vendas de produtos para tomadas de decisão baseado em analise de dados aplicando modelo de inteligência artificial

In [3]:
#import pandas as pd
#import seaborn as sns

In [5]:
# 📦 Imports
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

# 📥 Carregamento dos dados
# o codigo abaixo abre o dataset em um aqruivo excel.
df = pd.read_excel('dataset_vendasProduto.xlsx')
# 🗂️ Visualização inicial
df.head()


,data_da_venda,categoria,produto,fabricante,profissional_remunerado,cliente,email,status_nfce,n_da_nfce,codigo_de_acesso_sefaz,...,subtotal,desconto,total,total_pago_a_descontar_prof,motivo_desconto,observacoes,custo_medio_atual_do_produto,custo_medio_produto_vendido,valor_ultima_compra,tipo_de_tributacao_do_produto
0,2022-07-28,Coloração,"LP Inoa 5,1 60g",L'Oréal,Amanda,virginia rodrigues,NaN,Emitida,36069.0,'53220721975658000199650030000360691000012340,...,126.00,0,126.00,0,NaN,NaN,0.0,0.0,NaN,Tributado
1,2022-07-27,Coloração,"LP Inoa 10,1 60g",L'Oréal,Ana Lucia,VANESSA MATOS,NaN,Emitida,36046.0,'53220721975658000199650030000360461000012346,...,70.00,0,70.00,0,NaN,NaN,0.0,0.0,NaN,Tributado
2,2022-07-27,Oxidante,LP Ox Inoa 6% 20 vol 1000ml,L'Oréal,Ana Lucia,VANESSA MATOS,NaN,Emitida,36046.0,'53220721975658000199650030000360461000012346,...,29.80,0,29.80,0,NaN,NaN,0.0,0.0,0.0,Tributado
3,2022-07-27,Coloração,KN Tinta Color 5.3 60ml,Keune,Amanda,virginia rodrigues,NaN,Emitida,36058.0,'53220721975658000199650030000360581000012347,...,53.91,0,53.91,0,NaN,NaN,0.0,0.0,NaN,Tributado
4,2022-07-20,Shampoo,Wp SP Luxe Oil Keratin Protect - Sh 200ml,Wella,Will,Luciana Meireles,NaN,Emitida,35970.0,'53220721975658000199650030000359701000012346,...,79.90,0,79.90,0,NaN,NaN,0.0,0.0,NaN,Tributado


In [6]:
# 🧩 Verificando tipos de dados
df.info()
# 🔍 Verificando valores únicos
df['produto'].value_counts()
# 🔢 Verificando valores numéricos
df.describe()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28012 entries, 0 to 28011
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   data_da_venda                  28012 non-null  datetime64[ns]
 1   categoria                      28012 non-null  object        
 2   produto                        28012 non-null  object        
 3   fabricante                     28012 non-null  object        
 4   profissional_remunerado        28012 non-null  object        
 5   cliente                        28012 non-null  object        
 6   email                          10234 non-null  object        
 7   status_nfce                    28012 non-null  object        
 8   n_da_nfce                      27234 non-null  float64       
 9   codigo_de_acesso_sefaz         27227 non-null  object        
 10  preco_unitario                 28012 non-null  float64       
 11  qtde           

,data_da_venda,n_da_nfce,preco_unitario,qtde,subtotal,desconto,total,total_pago_a_descontar_prof,motivo_desconto,observacoes,custo_medio_atual_do_produto,custo_medio_produto_vendido,valor_ultima_compra
count,28012,27234.000000,28012.000000,28012.000000,28012.000000,28012.0,28012.000000,28012.000000,0.0,0.0,28012.000000,28012.000000,8737.000000
mean,2024-01-29 07:02:49.127516672,43601.850371,71.670330,1.150471,77.522953,0.0,77.521525,0.001428,NaN,NaN,29.254568,31.493000,93.787794
min,2022-05-02 00:00:00,34612.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000
25%,2023-04-24 00:00:00,39859.000000,30.000000,1.000000,38.000000,0.0,38.000000,0.000000,NaN,NaN,0.000000,0.000000,49.900000
50%,2024-02-03 00:00:00,44149.000000,59.900000,1.000000,61.630000,0.0,61.630000,0.000000,NaN,NaN,0.000000,0.000000,89.000000
75%,2024-11-26 00:00:00,47218.750000,95.000000,1.000000,99.900000,0.0,99.900000,0.000000,NaN,NaN,49.900000,49.900000,119.900000
max,2025-06-04 00:00:00,49978.000000,1800.000000,18.000000,1800.000000,0.0,1800.000000,40.000000,NaN,NaN,669.000000,1110.000000,669.000000
std,NaN,4281.770874,57.624318,0.657267,63.059019,0.0,63.060321,0.238995,NaN,NaN,56.781174,65.309627,65.457461


In [ ]:
# 🧹 Limpeza básica
df = df.dropna(subset=['total', 'qtde', 'preco_unitario', 'produto'])
# a linha acima remove linhas com valores ausentes nas colunas especificadas.



# 📅 Conversão de data
df['data_da_venda'] = pd.to_datetime(df['data_da_venda'], dayfirst=True)
# a linha acima converte a coluna 'data_da_venda' para o tipo datetime, assumindo que o dia vem antes do mês.
df['ano'] = df['data_da_venda'].dt.year
# a linha acima extrai o ano da coluna 'data_da_venda' e cria uma nova coluna 'ano'.
df['mes'] = df['data_da_venda'].dt.month
# a linha acima extrai o mês da coluna 'data_da_venda' e cria uma nova coluna 'mes'.
df['dia'] = df['data_da_venda'].dt.day
# a linha acima extrai o dia da coluna 'data_da_venda' e cria uma nova coluna 'dia'.
df['dia_semana'] = df['data_da_venda'].dt.dayofweek
# a linha acima extrai o dia da semana da coluna 'data_da_venda' e cria uma nova coluna 'dia_semana'.
# a coluna 'dia_semana' é um número de 0 (segunda-feira) a 6 (domingo).


# 🎯 Variável alvo
target = 'total'

# 🧠 Features numéricas e categóricas
features = ['qtde', 'preco_unitario', 'ano', 'mes', 'dia', 'dia_semana',
            'categoria', 'produto', 'fabricante', 'profissional_remunerado']

# 🔀 Codificar variáveis categóricas
for col in ['categoria', 'produto', 'fabricante', 'profissional_remunerado']:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

# 🧪 Separar dados
X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🚀 Treinar modelo LightGBM
model = lgb.LGBMRegressor(random_state=42, n_estimators=1000, learning_rate=0.05)
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=50, verbose=False)

# 📈 Previsões
y_pred = model.predict(X_test)

# 📊 Avaliação
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"✅ RMSE: {rmse:.2f}")
print(f"✅ R² Score: {r2:.2%}")

# 🔍 Importância das features
importancias = pd.DataFrame({'Feature': X.columns, 'Importância': model.feature_importances_})
importancias = importancias.sort_values(by='Importância', ascending=False)

# 📊 Plotar importância
plt.figure(figsize=(10,6))
sns.barplot(x='Importância', y='Feature', data=importancias)
plt.title('Importância das Variáveis - LightGBM')
plt.tight_layout()
plt.show()
